In [8]:
import pandas as pd
import numpy as np

In [9]:
import psycopg2

In [10]:
try:
    conn = psycopg2.connect("dbname=metacritic user=postgres")
except:
    print("cannot connect")

In [11]:
cur = conn.cursor()

In [12]:
import pandas.io.sql as sqlio

In [13]:
sql = "select * from critics;"
critics = sqlio.read_sql_query(sql, conn)

In [14]:
critics.head()

,id,name
0,a189f8ea-e791-4c81-bd22-490cdf4591b1,Logo
1,d5f89497-55c5-4c5c-9272-be319f04b087,The A.V. Club
2,f5ce11dc-eddd-415f-9e12-bb085014c5f6,Exclaim
3,cd9d6f62-d07b-49e8-af6a-0db0773672de,The 405
4,72cdb8d5-cf1e-4ada-8111-f42ddd3e6167,Kerrang!


In [15]:
sql = "select * from albums;"
albums = sqlio.read_sql_query(sql, conn)
#conn = None

In [16]:
albums.head()

,id,name,artist,genres,link,img_link,mc_rating,usr_rating
0,1463a7e4-5bad-482a-9f85-9ac53fce479f,2:54,2:54,"[Pop/Rock, Alternative/Indie Rock, Indie Rock]",/music/254/254,http://static.metacritic.com/images/products/m...,68.0,8.3
1,b3148743-3cc2-41ab-b6c9-67516c0b08eb,The Other I,2:54,"[Pop/Rock, Alternative/Indie Rock, Hard Rock]",/music/the-other-i/254,http://static.metacritic.com/images/products/m...,69.0,NaN
2,0ae002f0-f35c-43d2-ae14-123dcb1b62fa,Plays Music,33.3,"[Indie, Rock, Instrumental]",/music/plays-music/333,http://static.metacritic.com/images/products/m...,51.0,NaN
3,844ffd1e-c437-436f-8547-a57016da09d0,The Hard Way,213,[Rap],/music/the-hard-way/213,http://static.metacritic.com/images/products/m...,56.0,8.0
4,5cd77908-b875-4e9f-aa41-100c08627f08,Don't Tread On Me,311,"[Rock, Alternative]",/music/dont-tread-on-me/311,http://static.metacritic.com/images/products/m...,51.0,6.1


In [17]:
sql = "select * from ratings;"
ratings = sqlio.read_sql_query(sql, conn)

In [18]:
ratings.head()

,id,album_link,album_name,album_rating,critic_name,album_description,critic_id,album_id
0,6f0eaebb-2196-4e56-ab38-6b19d7ff7cd6,/music/modern-apprentice/ikara-colt,Modern Apprentice,90,Logo,"Quite simply, this is the most invigorating al...",a189f8ea-e791-4c81-bd22-490cdf4591b1,61e3191e-3177-4b15-828f-d2524bc0a6db
1,2d573dfc-2016-4e4d-a8e7-1edbe26ba08d,/music/auf-der-maur/auf-der-maur,Auf Der Maur,80,Logo,Though Auf Der Maur has run off with their blu...,a189f8ea-e791-4c81-bd22-490cdf4591b1,8edf003d-9237-4d06-aedd-9b914793aa7e
2,5db9090b-637e-4240-8eec-5faa117d98a1,/music/sung-tongs/animal-collective,Sung Tongs,90,Logo,This is what you get when you give an overacti...,a189f8ea-e791-4c81-bd22-490cdf4591b1,ca0f5b51-4cb4-4251-b8a3-b7fbd349d9c4
3,cb9f5d9c-1c01-4094-931f-d294860b7871,/music/devin-dazzle-and-the-neon-fever/felix-d...,Devin Dazzle And The Neon Fever,80,Logo,Felix Da Housecat's shift into the wastelands ...,a189f8ea-e791-4c81-bd22-490cdf4591b1,ead4e1f2-34c5-488b-9c2e-7af4689e3d6a
4,1687f02c-4ccf-4eee-808a-2c1fcdafc5e2,/music/achilles-heel/pedro-the-lion,Achilles Heel,80,Logo,"Bazan will never sound truly happy on record, ...",a189f8ea-e791-4c81-bd22-490cdf4591b1,13a7b7b2-855d-4cc2-8906-732488854414


In [19]:
rating_totals = pd.DataFrame(ratings.groupby('album_id')['album_rating'].count())
rating_totals.columns = ['rating_count']
rating_totals['album_rating'] = ratings.groupby('album_id')['album_rating'].mean()
rating_totals.head()

,rating_count,album_rating
album_id,,
000170c8-cc89-418f-85d6-99b2cddbfe2a,6,70.500000
00023c63-5cf0-496b-9fed-aedd51e258bd,11,74.363636
0002e56a-0c28-4e7d-a417-70ba6069674b,6,64.500000
000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,20,60.000000
0013b996-5b63-4dd3-aeea-d6a05b9f27ed,6,81.666667


In [20]:
crosstab = pd.pivot_table(data=ratings, values='album_rating', index='critic_id', columns='album_id')

In [21]:
crosstab.head()

album_id,000170c8-cc89-418f-85d6-99b2cddbfe2a,00023c63-5cf0-496b-9fed-aedd51e258bd,0002e56a-0c28-4e7d-a417-70ba6069674b,000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,0013b996-5b63-4dd3-aeea-d6a05b9f27ed,0016bb78-ca68-4364-bc22-34047f61ea04,001f620a-190a-405f-bc11-b1d8d95007eb,00216190-8202-46f3-9952-45b061802075,0023d132-670c-4ac5-8b83-d49b373ec595,0023d215-eae3-4d0e-855c-31562215be26,...,ffcb36a0-3597-4f41-b5e4-2f754b0d32fd,ffd2610f-6e51-418c-9a55-c9df19943795,ffdbe0bc-5319-468a-a971-db16b0ea21d3,ffe6671b-46d2-4d12-aff3-fa88323caef3,ffed69c0-a428-4fb3-bc17-3850dcb70e60,ffee18ab-7a95-4305-984c-4a24ee9497d7,fff1d03a-ff43-48de-993b-e02afb2355cf,fff91846-ec09-4cf2-9446-a58f032d2e44,fffbb171-a1df-418e-83d0-0ecd89a0828e,fffd8d2b-7f94-4d60-a043-ad91f06c68b9
critic_id,,,,,,,,,,,,,,,,,,,,,
06193874-b01f-4bde-be51-7e4ca2b595e0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,...,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,80.0
07c9b103-81e3-425e-bdb0-3ad0b968eff2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0a944835-ed0e-4778-b5b1-1e932970c57d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN
0c63e045-ccc6-453c-9aa4-23b33e6f263d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0eb4e4eb-710d-4335-b214-9be9e8e37383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN


In [23]:
album_name = input("Enter Your Album: ")

Enter Your Album: Funeral


In [24]:
album_name

'Funeral'

In [25]:
pivot_album_id = albums[albums['name']==album_name].id
print(pivot_album_id)
#TODO: get album id as string, not object, so it can be passed to next cell programmatically!

788    b5438c07-02b5-41c7-a78d-596b5ba8cc5e
Name: id, dtype: object


In [26]:
album_ratings = crosstab['b5438c07-02b5-41c7-a78d-596b5ba8cc5e']
albums[albums['id']=='b5438c07-02b5-41c7-a78d-596b5ba8cc5e']
album_ratings[album_ratings>=0]

critic_id
06193874-b01f-4bde-be51-7e4ca2b595e0     90.0
0eb4e4eb-710d-4335-b214-9be9e8e37383     76.0
0f74232b-d84d-4988-8de9-f2ad2fef1ede     60.0
133ea9b0-8f87-4234-a5a9-18a83d9bce5a     80.0
15704ba6-0890-4afa-a825-e04960031721     91.0
171aa7d3-6618-4578-b653-8ffdd3a5d616     90.0
18b4537e-ecd8-49ea-8523-9f2b6366515d     90.0
36c5471a-b29d-43f3-a469-aef4b36241cf    100.0
3e060ad3-c38e-448d-9245-0fb8d0add993     70.0
403b2b1e-a9fa-42d1-89dd-3c840d9025ad    100.0
406d6c62-f065-4617-9736-d3b2804a8e4e     80.0
440dfb93-96a7-4f0b-bb6e-cb68510668b2    100.0
49785ef6-e420-416a-b219-a1029e888522    100.0
4b0b8444-4cd6-4301-9251-546222119a2d     90.0
4db94751-4c3c-47b6-b5d6-2ebb752f5cd2     90.0
5267a0e4-0d2d-4a35-b815-c6db9aa867e3     97.0
6154ad7e-1a7f-487b-a65f-caae8a05da82     80.0
6e890ea9-d4b9-4976-9d6b-03163632a4be     90.0
85260d4f-2f68-406e-b747-ba554932d604     90.0
8af51262-5d50-4052-97e3-bc2b4138fd6d     70.0
8dd2934a-b766-4e5d-a7e6-0ca8dffd1e04    100.0
8f14ad6e-5a25-46e7-8655-

In [27]:
similar_to_album = crosstab.corrwith(album_ratings)

/Users/michael/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/michael/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


In [28]:
corr_album = pd.DataFrame(similar_to_album, columns=['PearsonR'])
corr_album.dropna(inplace=True)
corr_album.head()

,PearsonR
album_id,
000170c8-cc89-418f-85d6-99b2cddbfe2a,-0.007065
00023c63-5cf0-496b-9fed-aedd51e258bd,-0.384349
000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,-0.431797
0016bb78-ca68-4364-bc22-34047f61ea04,0.894427
001f620a-190a-405f-bc11-b1d8d95007eb,0.189476


In [29]:
#TODO: Sort by PearsonR, set minimum number of ratings (join)
corr_summary = corr_album.join(rating_totals)
corr_summary.head()

,PearsonR,rating_count,album_rating
album_id,,,
000170c8-cc89-418f-85d6-99b2cddbfe2a,-0.007065,6,70.500000
00023c63-5cf0-496b-9fed-aedd51e258bd,-0.384349,11,74.363636
000cbff7-17f1-4dd8-9cd9-5eb1157fbb95,-0.431797,20,60.000000
0016bb78-ca68-4364-bc22-34047f61ea04,0.894427,6,85.000000
001f620a-190a-405f-bc11-b1d8d95007eb,0.189476,10,71.400000


In [30]:
corr_summary[corr_summary['rating_count']>=25].sort_values('PearsonR', ascending=False).head(20)

,PearsonR,rating_count,album_rating
album_id,,,
b5438c07-02b5-41c7-a78d-596b5ba8cc5e,1.000000,33,86.969697
b663a6be-a1a8-4694-8904-3a57cd34d2ff,0.707450,28,75.821429
0784e981-72ab-4d05-8ce8-8944f203d10d,0.686086,28,77.535714
3479070b-80f6-4c75-8bdb-1a38ab3918a7,0.669224,33,72.424242
eceab1b5-894d-4a9a-abcf-3e77db0ba05d,0.668704,27,70.444444
9a3bf4a3-dd2a-4be3-9dd5-47aed059d2ed,0.668694,25,82.640000
3b8376ed-0d3a-4ce5-821c-16c980b7f985,0.645986,33,75.727273
4bd1b744-eb07-42af-83ab-628ab921af05,0.642853,26,72.884615
0f6c93cb-7636-4146-b3ee-ceffefb3752e,0.634001,34,84.088235


In [31]:
albums[albums['id']=='0784e981-72ab-4d05-8ce8-8944f203d10d']

,id,name,artist,genres,link,img_link,mc_rating,usr_rating
5075,0784e981-72ab-4d05-8ce8-8944f203d10d,I'm New Here,Gil Scott-Heron,"[Rap, R&B, Blues]",/music/im-new-here/gil-scott-heron,http://static.metacritic.com/images/products/m...,78.0,8.8
